In [1]:
from sqlalchemy import create_engine
import sqlite3
import pandas as pd

In [5]:
#Connecting to sqlite
conn = sqlite3.connect('sqlite_db_pythonsqlite.db')

In [12]:
sql_qry = "select * from Facilities"

fac_df = pd.read_sql_query(sql_qry, conn)
fac_df.head()

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000


## Part I ##

**Question 1** Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do.

In [8]:
sql_qry = "select distinct name from Facilities where membercost >=0 OR membercost is not NULL"

df1 = pd.read_sql_query(sql_qry, conn)
print(df1)

              name
0   Tennis Court 1
1   Tennis Court 2
2  Badminton Court
3     Table Tennis
4   Massage Room 1
5   Massage Room 2
6     Squash Court
7    Snooker Table
8       Pool Table


**Question 2** How many facilities do not charge a fee to members?

In [11]:
sql_qry = "select count(name) as `Number of Facilities - No Charge` from Facilities where membercost ==0 OR membercost is not NULL"

df2 = pd.read_sql_query(sql_qry, conn)
df2

,Number of Facilities - No Charge
0,9


**Question 3** Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question.

In [13]:
sql_qry = """SELECT name, membercost, monthlymaintenance 
            FROM Facilities F WHERE
            F.membercost < (.2*F.monthlymaintenance)
            """ 

df3 = pd.read_sql_query(sql_qry, conn)
df3

,name,membercost,monthlymaintenance
0,Tennis Court 1,5.0,200
1,Tennis Court 2,5.0,200
2,Badminton Court,0.0,50
3,Table Tennis,0.0,10
4,Massage Room 1,9.9,3000
5,Massage Room 2,9.9,3000
6,Squash Court,3.5,80
7,Snooker Table,0.0,15
8,Pool Table,0.0,15


**Question 4** Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator.

In [14]:
sql_qry = "SELECT * FROM Facilities WHERE facid IN (1, 5)"

df4 = pd.read_sql_query(sql_qry, conn)
df4

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,1,Tennis Court 2,5.0,25,8000,200
1,5,Massage Room 2,9.9,80,4000,3000


**Question 5** Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question.

In [17]:
sql_qry = """ SELECT name,
                    CASE WHEN monthlymaintenance > 100 THEN "expensive"
                    ELSE "cheap" END AS Label
            FROM Facilities """

df5 = pd.read_sql_query(sql_qry, conn)
df5

,name,Label
0,Tennis Court 1,expensive
1,Tennis Court 2,expensive
2,Badminton Court,cheap
3,Table Tennis,cheap
4,Massage Room 1,expensive
5,Massage Room 2,expensive
6,Squash Court,cheap
7,Snooker Table,cheap
8,Pool Table,cheap


**Question 6** You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution.

In [19]:
sql_qry = """
        SELECT firstname, surname
        FROM Members
        ORDER BY joindate DESC
        """

df6 = pd.read_sql_query(sql_qry, conn)
df6

,firstname,surname
0,Darren,Smith
1,Erica,Crumpet
2,John,Hunt
3,Hyacinth,Tupperware
4,Millicent,Purview
5,Henry,Worthington-Smyth
6,David,Farrell
7,Henrietta,Rumney
8,Douglas,Jones
9,Ramnaresh,Sarwin


**Question 7** Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name.

In [27]:
sql_qry = """
        SELECT DISTINCT m.firstname ||' ' || m.surname AS MemberName
        FROM Members m
            LEFT JOIN Bookings b
                ON m.memid = b.memid
            LEFT JOIN Facilities f
                ON b.facid = f.facid
        WHERE f.name LIKE '%Tennis Court%'
        """

df7 = pd.read_sql_query(sql_qry, conn)
df7

,MemberName
0,GUEST GUEST
1,Darren Smith
2,Tracy Smith
3,Tim Rownam
4,Janice Joplette
5,Gerald Butters
6,Burton Tracy
7,Nancy Dare
8,Tim Boothe
9,Ponder Stibbons


**Question 8** Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries.

In [52]:
# The per half-hour cost is not clear -- I'm assuming that the member's listed cost on the table is the price per half hour
# e.g. $5 is actually $10 for one slot. This indicates that member's cost needs to be 2 * membercost for each number of slots.
# i'm assuming that each reservation is also made for all the slots. 
# Therefore total costs is actually _slots*guestcost_ for guests and _slots*2*membercost_ for members
# there's an issue of members cost is listed in guest costs as well (e.g. ID != 0 but is listed 0 for memberscost)

sql_qry = """
        SELECT DISTINCT m.firstname ||' ' || m.surname AS MemberName, f.name as Facility,
                
                CASE WHEN m.memid != 0 AND f.membercost = 0 THEN b.slots*2*f.guestcost
                    WHEN m.memid != 0 AND f.membercost != 0 THEN b.slots*2*f.membercost
                    ELSE f.guestcost*b.slots END AS Cost

        FROM Members m
            LEFT JOIN Bookings b
                ON m.memid = b.memid
            LEFT JOIN Facilities f
                ON b.facid = f.facid

        WHERE  date(b.starttime) = '2012-09-14' AND 
        (CASE WHEN m.memid != 0 AND f.membercost = 0 THEN b.slots*2*f.guestcost
                    WHEN m.memid != 0 AND f.membercost != 0 THEN b.slots*2*f.membercost
                    ELSE f.guestcost*b.slots END) > 30
        ORDER BY Cost DESC           
        """

df8 = pd.read_sql_query(sql_qry, conn)
df8

,MemberName,Facility,Cost
0,GUEST GUEST,Massage Room 2,320.0
1,GUEST GUEST,Massage Room 1,160.0
2,GUEST GUEST,Tennis Court 2,150.0
3,Darren Smith,Badminton Court,93.0
4,Gerald Butters,Badminton Court,93.0
5,Ponder Stibbons,Badminton Court,93.0
6,Anna Mackenzie,Badminton Court,93.0
7,Jemima Farrell,Massage Room 1,79.2
8,GUEST GUEST,Tennis Court 1,75.0
9,GUEST GUEST,Tennis Court 2,75.0


**Question 9** This time, produce the same result as in Q8, but using a subquery.

In [50]:
sql_qry = """
          SELECT DISTINCT firstname ||' ' || surname AS MemberName, 
                  Facility, Cost
            FROM (SELECT m.firstname, m.surname, f.name AS Facility,
                
                    CASE WHEN b.memid != 0 AND f.membercost = 0 THEN b.slots*2*f.guestcost
                        WHEN b.memid != 0 AND f.membercost != 0 THEN b.slots*2*f.membercost
                        ELSE f.guestcost*b.slots END AS Cost

                FROM Members m
                    LEFT JOIN Bookings b
                        ON m.memid = b.memid
                    LEFT JOIN Facilities f
                        ON b.facid = f.facid

                WHERE  date(b.starttime) = '2012-09-14' AND 
                    (CASE WHEN b.memid != 0 AND f.membercost = 0 THEN b.slots*2*f.guestcost
                        WHEN b.memid != 0 AND f.membercost != 0 THEN b.slots*2*f.membercost
                        ELSE f.guestcost*b.slots END) > 30) as subquery
        order by Cost Desc  
            """

df9 = pd.read_sql_query(sql_qry, conn)
df9

,MemberName,Facility,Cost
0,GUEST GUEST,Massage Room 2,320.0
1,GUEST GUEST,Massage Room 1,160.0
2,GUEST GUEST,Tennis Court 2,150.0
3,Darren Smith,Badminton Court,93.0
4,Gerald Butters,Badminton Court,93.0
5,Ponder Stibbons,Badminton Court,93.0
6,Anna Mackenzie,Badminton Court,93.0
7,Jemima Farrell,Massage Room 1,79.2
8,GUEST GUEST,Tennis Court 1,75.0
9,GUEST GUEST,Tennis Court 2,75.0


## Part II ##

**Question 10** Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members!

In [58]:
sql_qry = """
            SELECT Facility, Revenue
            FROM (SELECT  f.name AS Facility,

                                SUM(CASE WHEN b.memid != 0 AND f.membercost = 0 THEN b.slots*2*f.guestcost
                                    WHEN b.memid != 0 AND f.membercost != 0 THEN b.slots*2*f.membercost
                                    ELSE f.guestcost*b.slots END) AS Revenue

                            FROM Bookings b
                                LEFT JOIN Facilities f
                                    ON b.facid = f.facid

            GROUP BY f.name) r
            WHERE Revenue < 10000
            ORDER BY Revenue Desc
            """
df10 = pd.read_sql_query(sql_qry, conn)
df10

,Facility,Revenue
0,Snooker Table,8840
1,Pool Table,8830
2,Table Tennis,8120


**Question 11** Produce a report of members and who recommended them in alphabetic surname,firstname order

In [79]:
sql_qry = """
            WITH ReferTbl AS (SELECT surname ||', ' || firstname AS Referral,
            memid FROM Members )
            SELECT r.Referral, m.surname ||', ' || m.firstname AS Member
            FROM ReferTbl r
                LEFT JOIN Members m
                        ON r.memid = m.recommendedby
            WHERE m.recommendedby != ''
            ORDER by r.Referral
            """
df11 = pd.read_sql_query(sql_qry, conn)
df11

,Referral,Member
0,"Bader, Florence","Sarwin, Ramnaresh"
1,"Baker, Timothy","Coplin, Joan"
2,"Butters, Gerald","Genting, Matthew"
3,"Farrell, Jemima","Baker, Timothy"
4,"Farrell, Jemima","Pinker, David"
5,"Genting, Matthew","Rumney, Henrietta"
6,"Jones, David","Jones, Douglas"
7,"Joplette, Janice","Dare, Nancy"
8,"Joplette, Janice","Jones, David"
9,"Purview, Millicent","Hunt, John"


**Question 12** Find the facilities with their usage by member, but not guests

In [80]:
sql_qry = """
            SELECT f.name, SUM(CASE WHEN memid != 0 THEN 1
                            ELSE 0 END) AS MemberUsage
            FROM Bookings b
                INNER JOIN Facilities f
                    on b.facid = f.facid
            GROUP BY f.name
            ORDER BY MemberUsage DESC
            """
df12 = pd.read_sql_query(sql_qry, conn)
df12

,name,MemberUsage
0,Pool Table,783
1,Snooker Table,421
2,Massage Room 1,421
3,Table Tennis,385
4,Badminton Court,344
5,Tennis Court 1,308
6,Tennis Court 2,276
7,Squash Court,195
8,Massage Room 2,27


**Question 13** Find the facilities usage by month, but not guests

In [95]:
sql_qry = """
            SELECT f.name, strftime('%m', b.starttime) AS Month,
                SUM(CASE WHEN b.memid != 0 THEN 1 ELSE 0 END) AS MemberUsage
            FROM Bookings b
                INNER JOIN Facilities f
                    on b.facid = f.facid
            GROUP BY f.name, Month
            ORDER BY Month, MemberUsage
            """
df13 = pd.read_sql_query(sql_qry, conn)
df13

,name,Month,MemberUsage
0,Massage Room 2,07,4
1,Squash Court,07,23
2,Tennis Court 2,07,41
3,Table Tennis,07,48
4,Badminton Court,07,51
5,Tennis Court 1,07,65
6,Snooker Table,07,68
7,Massage Room 1,07,77
8,Pool Table,07,103
9,Massage Room 2,08,9
